<a href="https://colab.research.google.com/github/HarshithaGM01/customRLEnv/blob/main/CustomRLEnv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys, platform, random
import numpy as np

print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())

SEED_271 = 271
random.seed(SEED_271)
np.random.seed(SEED_271)


Python: 3.12.12
Platform: Linux-6.6.105+-x86_64-with-glibc2.35


In [2]:
!pip -q install "gymnasium>=0.29" "stable-baselines3>=2.2.1" "shimmy>=1.3.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 5.0 MB/s eta 0:00:00


In [3]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np


In [4]:
class LongHorizonDecisionEnv(gym.Env):
    metadata = {"render_modes": []}
    def __init__(self, max_steps=200, seed=271):
      super().__init__()

      self.max_steps = max_steps
      self.current_step = 0

      self._rng = np.random.default_rng(seed)

      #observation space - we have [skill,energy,resources,confidence,market_noise]
      self.observation_space = spaces.Box(
          low=0.0,
          high=1.0,
          shape=(5,),
          dtype=np.float32
      )

      #Action space - 0: invest in a skill
      # 1: exploit current skill
      # 2: rest/recover
      # 3: take risky opportunity
      # 4: play safe
      self.action_space = spaces.Discrete(5)

      self.state = None

    def reset(self, seed=None, options=None):
      super().reset(seed=seed)

      if seed is not None:
        self._rng = np.random.default_rng(seed)

      self.current_step = 0

      skill = self._rng.uniform(0.1, 0.3)
      energy = self._rng.uniform(0.6, 0.9)
      resources = self._rng.uniform(0.4, 0.7)
      confidence = self._rng.uniform(0.3, 0.6)
      market_noise = self._rng.uniform(0.0, 1.0)

      self.state = np.array(
            [skill, energy, resources, confidence, market_noise],
            dtype=np.float32
        )

      return self.state, {}
